In [ ]:
!pip install scikit-video

In [ ]:
!pip install opencv-contrib-python

In [ ]:
import os
import json
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from numpy.linalg import lstsq
import skvideo
import skvideo.io
import skimage.io as skio
import tkinter
import skimage
# matplotlib.use('TkAgg')
import cv2

In [ ]:
#Read in the Video
def readVideo(mp4):
    vidcap = cv2.VideoCapture(mp4)
    success, image = vidcap.read()
    if success:
        cv2.imwrite("first_frame.jpg", image)
    if not os.path.isfile("./data.json"):
        frame_0 = plt.imread("first_frame.jpg")
        plt.imshow(frame_0)
        data = plt.ginput(21 + 35 + 10,timeout=0)
        plt.close()

        with open('data.json', 'w') as f:
            json.dump(data,f)
    else:
        # read the file
        with open('data.json') as f:
            points = [tuple(x) for x in json.load(f)]
        return points

file = "box.mp4"
json2d = "./data.json"
first_frame = "./first_frame.jpg"

def label3dpoints( json2d, imagename):
    with open(json2d) as f:
            points = [tuple(x) for x in json.load(f)]
    img = plt.imread(imagename)
    total_tuples = []
    if not os.path.isfile("./3ddata.json"):
        for j in range(7):
            for i in range(6):
                total_tuples.append((i, j , 2))

        for j in range(6, 0, -1):
            total_tuples.append((0, j, 1))

        for i in range(6):
            total_tuples.append((i, 0, 1))


        for j in range(6, 0, -1):
            total_tuples.append((0, j, 0))

        for i in range(6):
            total_tuples.append((i, 0, 0))



        for t in total_tuples:
            print(t)
        with open('3data.json', 'w') as f:
            json.dump(str(total_tuples),f)


def debug_data(json2d, imgname,  cmd = ""):
    if cmd == "json":
        with open(json2d) as f:
                points = [tuple(x) for x in json.load(f)]
        print(points[25], points[26], points[27])
        points.pop(25)
        with open('data.json', 'w') as f:
                json.dump(points,f)
    elif cmd == "plot_points":
        with open(json2d) as f:
            points = [x for x in json.load(f)]
        img = plt.imread(imgname)
        p = np.array(points)
        plt.imshow(img)
        plt.scatter(p[:,0], p[:,1],  c='r', s=5)
        plt.show()

# label3dpoints(json2d, first_frame)



In [ ]:
#draw bounding boxes
file = "box.mp4"
json2d = "./data.json"
json3d = "./3data.json"
first_frame = "./first_frame.jpg"
def createtrackers(first_frame, json2d, mp4file):
  #iterable of frames
  print(mp4file)
  video = skvideo.io.vread(mp4file)


  #initialize bounding boxes for the first frames
  with open(json2d) as f:
            points2d = np.array([x for x in json.load(f)])
  with open(json3d) as g:
          points3d = json.load(g)
  points3d = eval(points3d)

  #citation: https://www.youtube.com/watch?v=5yrwXR9rJ0c
  #[1] Draw bounding boxes for each point as [x, y, w, h]
  #spec says to draw a bounding box that 8 * 8, lets up to 10
  bbox_offset = 30
  tracker_list = []
  for x,y in points2d:
    bbox = (x - bbox_offset/2, y - bbox_offset/2, bbox_offset, bbox_offset)
    # tracker = cv2.legacy.TrackerCSRT_create()
    # tracker = cv2.legacy.TrackerBoosting_create()
    tracker = cv2.legacy.TrackerMedianFlow_create()
    tracker_list.append(tracker)
    ret = tracker.init(video[0], bbox)

  mapframeto3dpoints = {k: [] for k in range(len(video))}
  mapframeto2dpoints = {j: [] for j in range(len(video))}
  for frame_num in range(len(video)):
    frame = video[frame_num]

    for tracker_num in range(len(tracker_list)):
      found, bbox = tracker_list[tracker_num].update(frame)
      if found:
        topleft = (int(bbox[0]), int(bbox[1]))
        bottomright = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, topleft, bottomright, (0,0,255), 3)

        width = int(bbox[0] + bbox[2]/2)
        height = int(bbox[1] + bbox[3]/2)
        mapframeto2dpoints[frame_num].append([width, height])
        mapframeto3dpoints[frame_num].append(points3d[tracker_num])


    cv2.imwrite("video_frames/{}.jpg".format(frame_num), frame)
  return mapframeto2dpoints, mapframeto3dpoints
mapframeto2dpoints, mapframeto3dpoints = createtrackers(first_frame, json2d, file)



box.mp4


In [ ]:
!mkdir video_frames

In [ ]:
!rm -rf video_frames

In [ ]:
def createVideo(directory, num_frames):
  filename_list = []
  video_name = './video_frames/final_video.mp4'

  for file_num in range(0, num_frames + 1):
    filename_list.append("./video_frames/{num}.jpg".format(num = file_num))

  frame = cv2.imread('./video_frames/1.jpg')

  # setting the frame width, height width
  # the width, height of first image
  height, width, layers = frame.shape

  img_list = []

  print(filename_list)
  for f in filename_list:
    img = skio.imread(f)
    img_list.append(img)

  out = cv2.VideoWriter('flow.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, (width, height))

  for i in range(len(img_list)):
    out.write(img_list[i])
  out.release()

createVideo(os.listdir("./video_frames"), 242)

['./video_frames/0.jpg', './video_frames/1.jpg', './video_frames/2.jpg', './video_frames/3.jpg', './video_frames/4.jpg', './video_frames/5.jpg', './video_frames/6.jpg', './video_frames/7.jpg', './video_frames/8.jpg', './video_frames/9.jpg', './video_frames/10.jpg', './video_frames/11.jpg', './video_frames/12.jpg', './video_frames/13.jpg', './video_frames/14.jpg', './video_frames/15.jpg', './video_frames/16.jpg', './video_frames/17.jpg', './video_frames/18.jpg', './video_frames/19.jpg', './video_frames/20.jpg', './video_frames/21.jpg', './video_frames/22.jpg', './video_frames/23.jpg', './video_frames/24.jpg', './video_frames/25.jpg', './video_frames/26.jpg', './video_frames/27.jpg', './video_frames/28.jpg', './video_frames/29.jpg', './video_frames/30.jpg', './video_frames/31.jpg', './video_frames/32.jpg', './video_frames/33.jpg', './video_frames/34.jpg', './video_frames/35.jpg', './video_frames/36.jpg', './video_frames/37.jpg', './video_frames/38.jpg', './video_frames/39.jpg', './video_

In [ ]:
video = skvideo.io.vread(file)

In [ ]:
#Citation: https://docs.opencv.org/3.4/d7/d53/tutorial_py_pose.html
def draw(img, imgpts):
    imgpts = imgpts[:, :2].astype(np.int)
    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img



def createCubeVideo(video, mapframeto2dpoints, mapframeto3dpoints, debug = False):
  img_list = []
  frame1 = np.array(video[0])
  height, width, layers = frame1.shape
  out = cv2.VideoWriter('project_cube.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, (width, height))
  cubecorners = np.array([[2,2,2,1], [2,3,2,1], [3,3,2,1], [3,2,2,1],
                          [2,2,3,1],[2,3,3,1],[3,3,3,1],[3,2,3,1]]).T
  for frame_index in range(len(video)):
    #get the
    frame = video[frame_index]
    points2d = np.array(mapframeto2dpoints[frame_index])
    points3d = np.array(mapframeto3dpoints[frame_index])

    points2d = np.hstack((points2d, np.ones((points2d.shape[0], 1))))
    points3d = np.hstack((points3d, np.ones((points3d.shape[0], 1))))

    projection_matrix = lstsq(points3d, points2d, rcond=None)[0]

    if debug:
      print("POINTS2D SHAPE")
      print(points2d.shape)

      print("POINTS3D SHAPE")
      print(points3d.shape)

      print("POINTS MATRIX SHAPEb")
      print(projection_matrix.shape)


    image_coords = np.matmul(projection_matrix.T, cubecorners)
    image_coords /= image_coords[2]

    boxed_image = draw(frame, image_coords.T)

    img = np.array(boxed_image)
    img_list.append(img)

    if debug:
       print(boxed_image.shape)
       print(image_coords.shape)

  for i in range(len(img_list)):
    frame = cv2.cvtColor(img_list[i], cv2.COLOR_BGR2RGB)
    out.write(frame)
  out.release()





createCubeVideo(video, mapframeto2dpoints, mapframeto3dpoints)


<ipython-input-26-a9a6102cbcec>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  imgpts = imgpts[:, :2].astype(np.int)


In [ ]:
!mkdir cube_frame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r "video_frames" "/content/drive/MyDrive/CS180_Final_Project"

In [ ]:
file = "box.mp4"
json2d = "./data.json"
json3d = "./3data.json"
first_frame = "./first_frame.jpg"
def drawLines(first_frame):
  with open(json2d) as f:
          points2d = np.array([x for x in json.load(f)])
  with open(json3d) as g:
          points3d = json.load(g)
  points3d = eval(points3d)
  origin = (0,0,0)
  yaxis = (0,6,0)
  xaxis = (5,0,0)
  zaxis = (0,0,2)

  o = []
  x = []
  y = []
  z = []
  for i in zip(points3d, points2d):
    if i[0] == origin:
      o = np.array(i[1]).astype(int)
    elif i[0] == yaxis:
      y = np.array(i[1]).astype(int)
    elif i[0] == xaxis:
      x = np.array(i[1]).astype(int)
    elif i[0] == zaxis:
      z = np.array(i[1]).astype(int)
  img = cv2.arrowedLine(first_frame, tuple(o), tuple(x), (255,0,0))
  img = cv2.arrowedLine(img, tuple(o), tuple(y), (0,255,0), 5)
  img = cv2.arrowedLine(img, tuple(o), tuple(z), (0,0,255), 5)
  skio.imsave("./pose.jpg", img)


img = plt.imread(first_frame)
drawLines(img)
  #